# Основы машинного обучения с приложениями в финансах

## Практическое задание «Разработка PD модели»


### Общая информация
Необходимо разработать модель предсказания дефолта. Даны финансовые данные 32 195 компаний (выручка, активы, пассивы и т.д). Результат будет оцениваться на скрытой выборке из 200 компаний по метрике качества `F1-macro` ([документация](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score)).

#### Правила
Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

#### Формат сдачи
Заполненный ноутбук с выводом ячеек отправлять на почту vbaklanova@hse.ru. Файл необходимо назвать в формате `ДЗ_Фамилия_Имя.ipynb`.

### Импортируем библиотеки

In [227]:
import pandas as pd
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

### Подгружаем данные

In [228]:
train_df = pd.read_csv("train.csv")
test_df  = pd.read_csv('test.csv')
desc_df  = pd.read_csv('data-desc.csv')

Описание данных

In [229]:
desc_df

,field,desc_eng,desc_rus
0,ar_revenue,Sales revenue (at the end of the last year),Выручка от реализации (на конец последнего года)
1,ar_total_expenses,Total expenses for the last year,Общие расходы за последний год
2,ar_sale_cost,Cost of sales (at the end of the last year),Себестоимость продаж (на конец последнего года)
3,ar_selling_expenses,Selling expenses (at the end of the last year),Коммерческие расходы (на конец последнего года)
4,ar_management_expenses,Administrative expenses (at the end of the las...,Управленческие расходы (на конец последнего года)
5,ar_sale_profit,Profit from sales (at the end of the last year),Прибыль от реализации (на конец последнего года)
6,ar_balance_of_rvns_and_expns,Balance of income and expenses (at the end of ...,Баланс доходов и расходов (на конец последнего...
7,ar_profit_before_tax,Gross Profit (end of last year),Валовая прибыль (на конец последнего года)
8,ar_taxes,Current income tax (at the end of the last year),Текущий на налог на прибыль (на конец последне...
9,ar_other_profit_and_losses,Other income and expenses (at the end of the l...,Прочие доходы и расходы (на конец последнего г...


Семплы и размеры выборок

In [230]:
train_df.head()

,record_id,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,...,adr_actual_age,head_actual_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg,default_12m
0,7,52996000.0,48344000.0,48344000.0,0.0,0.0,4652000.0,-2608000.0,2044000.0,846000.0,...,3,3,3,[1-100],50000.0,2,0,0,0,0
1,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12,12,12,[1-100],10000.0,5,0,0,0,0
2,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,28,28,28,[1-100],10000.0,2,0,0,0,0
3,22,32049000.0,29568000.0,29568000.0,0.0,0.0,2481000.0,-260000.0,2221000.0,302000.0,...,8,8,8,[1-100],300000.0,1,0,0,0,0
4,23,16458000.0,14935000.0,14935000.0,0.0,0.0,1523000.0,-274000.0,1249000.0,538000.0,...,27,27,27,[1-100],10000.0,2,0,0,0,0


In [231]:
train_df.shape

(32195, 37)

In [232]:
test_df.head()

,record_id,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,...,ogrn_age,adr_actual_age,head_actual_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg
0,40,125432000.0,122655000.0,122655000.0,0.0,0.0,2777000.0,-63000.0,2714000.0,579000.0,...,132,15,15,15,(100-500],10000.0,2,0,0,0
1,701,22458000.0,20201000.0,20201000.0,0.0,0.0,2257000.0,-749000.0,1508000.0,302000.0,...,76,29,29,29,[1-100],11000.0,1,0,0,0
2,1497,33616000.0,20742000.0,20742000.0,0.0,0.0,12874000.0,-2949000.0,9925000.0,0.0,...,139,3,3,3,[1-100],900.0,9,0,0,0
3,1687,12744000.0,6201000.0,6201000.0,0.0,0.0,6543000.0,-3898000.0,2645000.0,0.0,...,135,24,24,24,[1-100],10000.0,1,0,0,0
4,2033,49307000.0,43964000.0,43964000.0,0.0,0.0,5343000.0,-1795000.0,3548000.0,0.0,...,140,25,25,25,[1-100],11550.0,1,0,0,0


In [233]:
test_df.shape

(200, 36)

Количество разных типов признаков

In [234]:
train_df.dtypes.value_counts()

float64    25
int64      11
object      1
Name: count, dtype: int64

Уровень дефолт-рейта в выборке = 6,45%:

In [235]:
train_df['default_12m'].values.mean()

0.06448206243205466

32195 уникальных наблюдений:

In [236]:
train_df.record_id.nunique()

32195

### Строим baseline-модель c 3 признаками

In [237]:
features = ['bus_age','ul_capital_sum', 'ul_founders_cnt']

In [238]:
features_df = train_df.loc[:,features]
features_df['default_12m'] = train_df['default_12m']

In [239]:
features_df.head()

,bus_age,ul_capital_sum,ul_founders_cnt,default_12m
0,198,50000.0,2,0
1,256,10000.0,5,0
2,33,10000.0,2,0
3,185,300000.0,1,0
4,220,10000.0,2,0


### Построим карту корреляций

In [240]:
features_df.corr()

,bus_age,ul_capital_sum,ul_founders_cnt,default_12m
bus_age,1.000000,0.065668,0.209176,-0.109315
ul_capital_sum,0.065668,1.000000,0.024085,-0.019289
ul_founders_cnt,0.209176,0.024085,1.000000,-0.023077
default_12m,-0.109315,-0.019289,-0.023077,1.000000


### Валидация результата

In [241]:
X_train, X_test, y_train, y_test = train_test_split(features_df.drop('default_12m',axis=1),
                                                    features_df['default_12m'],
                                                    random_state=42, shuffle=True)

logit = LogisticRegression(C=1,solver='lbfgs',max_iter=1000)
cv_baseline = cross_val_score(logit, X_train, y_train, cv=5, scoring='f1_macro').mean()
cv_baseline

0.48338646470019536

F1_macro baseline-модели = 48.34%

### Обучение модели на всех данных

In [242]:
logit.fit(features_df.drop('default_12m',axis=1), features_df['default_12m'])

LogisticRegression(C=1, max_iter=1000)

### Формирование файла с ответом

In [249]:
answ_df = test_df[['record_id']]

In [250]:
answ_df['predict'] = logit.predict(test_df.loc[:,features])

In [251]:
answ_df.to_csv('submit.csv',index=False)